In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow. keras import layers, regularizers
import os
import pandas as pd
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist. load_data()
x_train = x_train. reshape (-1, 28, 28, 1) .astype ("float32") / 255.0
x_test = x_test. reshape (-1, 28, 28, 1) .astype ("float32") / 255.0

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
#CNN batchnorm relu for this structure we can make a block for this

#it makes debugging very easy

In [4]:
class CNNBlock(layers.Layer):

  def __init__(self, out_channels, kernel_size=3):
    super (CNNBlock, self).__init__()
    self.conv = layers. Conv2D(out_channels, kernel_size, padding='same')
    self.bn = layers. BatchNormalization ()

  def call (self, input_tensor, training=False) : #in call method we give tensor input, bascically define the path of data to go from where to where
    x = self. conv (input_tensor)
    x = self.bn(x, training=training)
    x = tf.nn. relu(x)
    return x

In [5]:
model = keras. Sequential(
[
    CNNBlock (32),
    CNNBlock (64) ,
    CNNBlock (128),
    layers .Flatten (),
    layers. Dense (10),
]
)

In [6]:
model. compile (optimizer=keras.optimizers. Adam (),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics= ["accuracy"],
)
model.fit (x_train, y_train, batch_size=64, epochs=3, verbose=2)
model. evaluate (x_test, y_test, batch_size=64, verbose=2)

Epoch 1/3
938/938 - 18s - loss: 0.5850 - accuracy: 0.9463 - 18s/epoch - 19ms/step
Epoch 2/3
938/938 - 11s - loss: 0.0862 - accuracy: 0.9822 - 11s/epoch - 11ms/step
Epoch 3/3
938/938 - 11s - loss: 0.0333 - accuracy: 0.9901 - 11s/epoch - 11ms/step
157/157 - 1s - loss: 0.0462 - accuracy: 0.9861 - 1s/epoch - 7ms/step


[0.04615749791264534, 0.9861000180244446]

In [7]:
#building similar to resnet

In [8]:
class ResBlock(layers.Layer):
  def __init__(self,channel):
    super(ResBlock,self).__init__()
    self.cnn1=CNNBlock(channel[0])
    self.cnn2=CNNBlock(channel[1])
    self.cnn3=CNNBlock(channel[2])
    self.pooling=layers.MaxPooling2D()

    #we create identity mapping fir skip connections
    self.identity_mapping=layers.Conv2D(channel[1],
                                        kernel_size=1, #we are not doing any convo here but only want to change shape so that can be added
                                        padding='valid')

  def call(self, input_tensor, training=False):
    x=self.cnn1(input_tensor, training=training)
    x=self.cnn2(x,training=training)
    x=self.cnn3(
        x+self.identity_mapping(input_tensor), #adding can only be done when shape is same, so need to take care of that before adding
        training=training)
    return self.pooling(x)


In [9]:
class ResNetLike(keras.Model):
  def __init__(self,num_classes=10):
    super(ResNetLike,self).__init__()
    self.block1=ResBlock([32,32,64])
    self.block2=ResBlock([64,64,128])
    self.block3=ResBlock([128,128,256])

    self.pooling=layers.GlobalAveragePooling2D()
    self.classifier=layers.Dense(num_classes)

  def call(self, input_tensor, training=False): #when we training or fit it make training=True and when evaluate then False
    x=self.block1(input_tensor, training=training)
    x=self.block2(x, training=training)
    x=self.block3(x, training=training)
    x=self.pooling(x)
    x=self.classifier(x)

    return x

In [10]:
model=ResNetLike(num_classes=10)
# model.summary() cannot as not specified input shape, we can only see summary after giving first input

In [18]:
model. compile (optimizer=keras.optimizers. Adam (),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics= ["accuracy"],
)
model.fit (x_train, y_train, batch_size=64, epochs=10, verbose=2)
model. evaluate (x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
938/938 - 21s - loss: 0.0836 - accuracy: 0.9760 - 21s/epoch - 22ms/step
Epoch 2/10
938/938 - 14s - loss: 0.0323 - accuracy: 0.9900 - 14s/epoch - 14ms/step
Epoch 3/10
938/938 - 13s - loss: 0.0257 - accuracy: 0.9917 - 13s/epoch - 14ms/step
Epoch 4/10
938/938 - 13s - loss: 0.0211 - accuracy: 0.9935 - 13s/epoch - 14ms/step
Epoch 5/10
938/938 - 13s - loss: 0.0187 - accuracy: 0.9942 - 13s/epoch - 14ms/step
Epoch 6/10
938/938 - 13s - loss: 0.0162 - accuracy: 0.9952 - 13s/epoch - 14ms/step
Epoch 7/10
938/938 - 13s - loss: 0.0144 - accuracy: 0.9954 - 13s/epoch - 14ms/step
Epoch 8/10
938/938 - 14s - loss: 0.0133 - accuracy: 0.9958 - 14s/epoch - 14ms/step
Epoch 9/10
938/938 - 13s - loss: 0.0115 - accuracy: 0.9963 - 13s/epoch - 14ms/step
Epoch 10/10
938/938 - 14s - loss: 0.0096 - accuracy: 0.9968 - 14s/epoch - 14ms/step
157/157 - 1s - loss: 0.0224 - accuracy: 0.9928 - 1s/epoch - 7ms/step


[0.022405853495001793, 0.9927999973297119]

In [12]:
model.summary()

Model: "res_net_like"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block (ResBlock)        multiple                  28640     
                                                                 
 res_block_1 (ResBlock)      multiple                  152896    
                                                                 
 res_block_2 (ResBlock)      multiple                  608896    
                                                                 
 global_average_pooling2d (  multiple                  0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_1 (Dense)             multiple                  2570      
                                                                 
Total params: 793002 (3.03 MB)
Trainable params: 791210 (3.02 MB)
Non-trainable params: 1792 (7.00 KB)
_________________

In [13]:
#summary is giving multiple so need to correct that

In [16]:
class ResNetLike(keras.Model):
  def __init__(self,num_classes=10):
    super(ResNetLike,self).__init__()
    self.block1=ResBlock([32,32,64])
    self.block2=ResBlock([64,64,128])
    self.block3=ResBlock([128,128,256])

    self.pooling=layers.GlobalAveragePooling2D()
    self.classifier=layers.Dense(num_classes)

  def call(self, input_tensor, training=False):
    x=self.block1(input_tensor, training=training)
    x=self.block2(x, training=training)
    x=self.block3(x, training=training)
    x=self.pooling(x)
    x=self.classifier(x)
    return x

  #write this
  def model(self):
    x=layers.Input(shape=(28,28,1))
    return keras.Model(inputs=[x], outputs=[self.call(x)])



In [17]:
model=ResNetLike(num_classes=10)
model.model().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block_6 (ResBlock)      (None, 14, 14, 64)        28640     
                                                                 
 res_block_7 (ResBlock)      (None, 7, 7, 128)         152896    
                                                                 
 res_block_8 (ResBlock)      (None, 3, 3, 256)         608896    
                                                                 
 global_average_pooling2d_2  (None, 256)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                             